In [0]:
ITSM_df = pd.read_csv(io.StringIO(uploaded['ITSM_data - ITSM_data (2).csv'].decode('utf-8')))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
ITSM_df.head()

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,Category,KB_number,Alert_Status,No_of_Reassignments,Open_Time,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,0.601292,incident,KM0000553,closed,26.0,05-02-2012 13:32,NaN,04-11-2013 13:50,04-11-2013 13:51,"3,87,16,91,111",Other,1.0,SD0000007,2.0,NaN,NaN
1,1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,0.415050,incident,KM0000611,closed,33.0,12-03-2012 15:44,02-12-2013 12:31,02-12-2013 12:36,02-12-2013 12:36,"4,35,47,86,389",Software,1.0,SD0000011,1.0,NaN,NaN
2,2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NaN,0.517551,request for information,KM0000339,closed,3.0,29-03-2012 12:36,NaN,13-01-2014 15:12,13-01-2014 15:13,"4,84,31,19,444",No error - works as designed,1.0,SD0000017,NaN,NaN,NaN
3,3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4.0,0.642927,incident,KM0000611,closed,13.0,17-07-2012 11:49,NaN,14-11-2013 09:31,14-11-2013 09:31,"4,32,18,33,333",Operator error,1.0,SD0000025,NaN,NaN,NaN
4,4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4.0,0.345258,incident,KM0000611,closed,2.0,10-08-2012 11:01,NaN,08-11-2013 13:55,08-11-2013 13:55,"3,38,39,03,333",Other,1.0,SD0000029,NaN,NaN,NaN


In [0]:
import datetime
import pandas as pd
import numpy as np
from collections import Counter


In [0]:
Counter(ITSM_df.Category).most_common(5)

[('incident', 37748),
 ('request for information', 8846),
 ('complaint', 11),
 ('request for change', 1)]

In [0]:
ITSM_df.Close_Time[3]

'14-11-2013 09:31'

In [0]:
ITSM_df1 = ITSM_df[ITSM_df.Priority.isnull()==False]

In [0]:
ITSM_df1.head()

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,Category,KB_number,Alert_Status,No_of_Reassignments,Open_Time,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,0.601292,incident,KM0000553,closed,26.0,05-02-2012 13:32,NaN,04-11-2013 13:50,04-11-2013 13:51,"3,87,16,91,111",Other,1.0,SD0000007,2.0,NaN,NaN
1,1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,0.415050,incident,KM0000611,closed,33.0,12-03-2012 15:44,02-12-2013 12:31,02-12-2013 12:36,02-12-2013 12:36,"4,35,47,86,389",Software,1.0,SD0000011,1.0,NaN,NaN
3,3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4.0,0.642927,incident,KM0000611,closed,13.0,17-07-2012 11:49,NaN,14-11-2013 09:31,14-11-2013 09:31,"4,32,18,33,333",Operator error,1.0,SD0000025,NaN,NaN,NaN
4,4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4.0,0.345258,incident,KM0000611,closed,2.0,10-08-2012 11:01,NaN,08-11-2013 13:55,08-11-2013 13:55,"3,38,39,03,333",Other,1.0,SD0000029,NaN,NaN,NaN
5,5,WBA000124,application,Web Based Application,WBS000088,IM0000013,Closed,4,4,4.0,0.006676,incident,KM0000611,closed,4.0,10-08-2012 11:27,NaN,08-11-2013 13:54,08-11-2013 13:54,"3,38,34,36,944",Other,1.0,SD0000031,NaN,NaN,NaN


In [0]:
ITSM_df1.shape

(45226, 26)

In [0]:
Counter(ITSM_df1.Priority)

Counter({1.0: 3, 2.0: 697, 3.0: 5323, 4.0: 22717, 5.0: 16486})

In [0]:
ITSM_df2=ITSM_df1[ITSM_df1.Close_Time>ITSM_df1.Open_Time]

In [0]:
ITSM_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39232 entries, 6 to 46605
Data columns (total 26 columns):
Unnamed: 0                    39232 non-null int64
CI_Name                       39232 non-null object
CI_Cat                        39138 non-null object
CI_Subcat                     39138 non-null object
WBS                           39232 non-null object
Incident_ID                   39232 non-null object
Status                        39232 non-null object
Impact                        39232 non-null object
Urgency                       39232 non-null object
Priority                      39232 non-null float64
number_cnt                    39232 non-null float64
Category                      39232 non-null object
KB_number                     39232 non-null object
Alert_Status                  39232 non-null object
No_of_Reassignments           39232 non-null float64
Open_Time                     39232 non-null object
Reopen_Time                   1653 non-null object
Resolv

In [0]:
selected =ITSM_df.loc[:,['CI_Subcat','WBS','Category','Priority']]
selected.dropna(inplace=True)
Counter(selected.Priority)

Counter({1.0: 3, 2.0: 689, 3.0: 5310, 4.0: 22689, 5.0: 16427})

In [0]:
selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45118 entries, 0 to 46605
Data columns (total 4 columns):
CI_Subcat    45118 non-null object
WBS          45118 non-null object
Category     45118 non-null object
Priority     45118 non-null float64
dtypes: float64(1), object(3)
memory usage: 1.7+ MB


In [0]:
selected = selected[selected.Priority!=1]

In [0]:
selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45115 entries, 0 to 46605
Data columns (total 4 columns):
CI_Subcat    45115 non-null object
WBS          45115 non-null object
Category     45115 non-null object
Priority     45115 non-null float64
dtypes: float64(1), object(3)
memory usage: 1.7+ MB


In [0]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
selected.CI_Subcat = enc.fit_transform(X.CI_Subcat)
selected.Category = enc.fit_transform(X.Category)
selected.WBS = selected.WBS.str[4:].astype('int')

In [0]:
Counter(selected.Category)

Counter({0: 11, 1: 36316, 2: 1, 3: 8787})

In [0]:
X = selected.loc[:,['CI_Subcat','WBS','Category',]]
y = selected.Priority

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,random_state=15)


In [0]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)


[[ 110    4   40   18]
 [  43  824  350  111]
 [  53  154 5189  276]
 [   8   44  774 3281]]


0.8337618583207731

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,accuracy_score
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

         2.0       0.51      0.64      0.57       172
         3.0       0.80      0.62      0.70      1328
         4.0       0.82      0.91      0.86      5672
         5.0       0.89      0.80      0.84      4107

    accuracy                           0.83     11279
   macro avg       0.76      0.74      0.74     11279
weighted avg       0.84      0.83      0.83     11279



In [0]:

model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[ 108    6   40   18]
 [  42  830  343  113]
 [  52  158 5190  272]
 [   8   42  770 3287]]


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8347371220852913